# TF-IDF
La tarea a resolver es que dada una query regrese el subtítulo más parecido, para esto pensamos en utilizar TF-IDF porque este método permite recuperar palabras clave a través de varios documentos, lo anterior puede utilizarse para hacer una búsqueda y con esas palabras clave asignar un score, a manera de que los primeros resultados sean más parecidos a la query de entrada, adicionalmente se pueden utilizar estos valores para agrupar los videos (en otros notebooks se hará este análisis).
Haremos el cálculo de TF-IDF de dos maneras:
- **Por cada subtítulo**: es decir que tomaremos cada subtítulo como un documento, esto porque a la hora de recuperar la query es más probable que si la query está en corpus la devuelva
- **Por vídeo**: aquí cada vídeo es un documento, y como el caso anterior podría ser que si regrese la query pero esta manera se enfoca más en regresar el subtítulo más parecido a la query, donde ese subtítulo además representa al documento, es decir que guarda más información en el resultado que nos devuelve, pero por lo mismo es más probable que no entiende la query tal cual si está es todo un subtítulo y no fragmentos, es decir que este método funciona mejor cuando se le da una palabra, mientras que el anterior funciona mejor para frases


In [ ]:
import pickle
from tqdm import tqdm 
from operator import itemgetter
#Para el corpus
import sys
sys.path.append('..')
from src.data import Data 
#Para calcular TF-IDF
from src.tfidf import TFIDF

Cargamos el corpus limpio

In [2]:
videos = Data("../pkl/clean_videos.pkl")
videosxO = Data("../pkl/clean_videos.pkl")
videos.corpus[0][0]

{'id': 'L9YhoRatRzE',
 'original_title': 'Siempre Fui Yo | Adelanto | Disney+',
 'subtitles': [{'start': '0.13',
   'dur': '3.77',
   'text': ['tu', 'papá', 'tuvo', 'un', 'accidente']},
  {'start': '12.5', 'dur': '5.939', 'text': ['te', 'recuerdo', 'que', 'está']},
  {'start': '15.59', 'dur': '5.339', 'text': ['aquí']},
  {'start': '18.439',
   'dur': '6.361',
   'text': ['estaba', 'como', 'rabioso', 'con']},
  {'start': '20.929',
   'dur': '7.65',
   'text': ['especial', 'con', 'lucas', 'martín']},
  {'start': '24.8',
   'dur': '3.779',
   'text': ['necesito', 'saber', 'qué', 'fue', 'lo', 'que', 'pasó']},
  {'start': '29.42', 'dur': '2.479', 'text': ['aplausos']}]}

In [3]:
videos.get_all_subtitles()
videosxO.get_all_subtitles()
videos.all_subtitles[5:10]

100%|██████████| 9/9 [00:00<00:00, 18.07it/s]


[['necesito', 'saber', 'qué', 'fue', 'lo', 'que', 'pasó'],
 ['aplausos'],
 ['decían', 'que', 'no', 'era', 'para', 'mí'],
 ['imposible'],
 ['que', 'nunca', 'lo', 'lograríamos']]

Obtenemos las frecuencias de cada palabra en todo el corpus

In [4]:
term_list = videos.get_frequencies()
term_list

,Frequency
Token,
de,217897
que,167644
la,128609
y,116389
el,105011
...,...
nananana,1
nía,1
llull,1


## TF-IDF tomando cada subtítulo como un documento

Calculamos el IDF de cada termino dentro de los documentos

In [4]:
tfidf_objxO = TFIDF(videosxO)

In [ ]:
tfidf_objxO.get_idf("../pkl/term_listxOracion.pkl", document=False)

In [5]:
tfidf_objxO.term_list = "../pkl/term_listxOracion.pkl"
tfidf_objxO.term_list[100:110]

,Frequency,idf
Token,,
empapando,1,19.043465
mpesa,1,19.043465
desacelerará,1,19.043465
enron,1,19.043465
chancay,1,19.043465
adquisitiva,1,19.043465
reelecto,1,19.043465
buje,1,19.043465
tergiversadas,1,19.043465


Nuestros documentos serán oraciones

In [ ]:
tfidf_objxO.get_tfidf("../pkl/tfidfxOracion.pkl", document=False)

In [7]:
tfidf_objxO.tfidf = "../pkl/tfidfxOracion.pkl"
tfidf_objxO.tfidf[0:5]

[{'tu': 22108.738225139823,
  'papá': 2083.0025225599124,
  'tuvo': 2992.536775205195,
  'un': 100312.7542205603,
  'accidente': 871.2425900002974},
 {'te': 37241.07464552038,
  'recuerdo': 1686.5502163091887,
  'que': 159589.1466637399,
  'está': 35915.05555718361},
 {'aquí': 24448.057573994738},
 {'estaba': 10512.081302027222,
  'como': 60085.75580795781,
  'rabioso': 50.164611432806424,
  'con': 69495.68171121314},
 {'especial': 3815.105651978379,
  'con': 69495.68171121314,
  'lucas': 79.3677011885693,
  'martín': 318.46288535642265}]

Agregamos los vectores tfidf a cada subtítulo dentro del corpus

In [8]:
tfidf_objxO.get_tfid_vectors(document=False)

Longitud de la oración más larga:  56


100%|██████████| 9/9 [00:04<00:00,  1.82it/s]


In [9]:
tfidf_objxO.data.corpus[0][0]

{'id': 'L9YhoRatRzE',
 'original_title': 'Siempre Fui Yo | Adelanto | Disney+',
 'subtitles': [{'start': '0.13',
   'dur': '3.77',
   'text': ['tu', 'papá', 'tuvo', 'un', 'accidente'],
   'tfid': [22108.738225139823,
    2083.0025225599124,
    2992.536775205195,
    100312.7542205603,
    871.2425900002974,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0]},
  {'start': '12.5',
   'dur': '5.939',
   'text': ['te', 'recuerdo', 'que', 'está'],
   'tfid': [37241.07464552038,
    1686.5502163091887,
    159589.1466637399,
    35915.05555718361,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0

## TF-IDF tomando cada vídeo como un documento

Calculamos el IDF de cada termino dentro de los documentos

In [10]:
tfidf_obj = TFIDF(videos)

In [ ]:
tfidf_obj.get_idf("../pkl/term_list.pkl")

In [11]:
tfidf_obj.term_list = "../pkl/term_list.pkl" 
tfidf_obj.term_list[100:110]

,Frequency,idf
Token,,
sólo,4461,1.284577
ejemplo,4431,1.277094
va,4387,1.114891
amigos,4357,1.510368
tiempo,4267,0.889507
donde,4216,0.994292
muchos,4108,1.164326
parte,4054,1.081919
estos,4004,1.047863


Nuestros documentos serán los subtítulos de todo el video

In [ ]:
tfidf_obj.get_tfidf("../pkl/tfidf.pkl")

In [13]:
tfidf_obj.tfidf = "../pkl/tfidf.pkl"
tfidf_obj.tfidf["L9YhoRatRzE"]

{'tu': 1418.7040158067475,
 'papá': 413.2296042991872,
 'tuvo': 433.0538386324496,
 'un': 1977.6454572285645,
 'accidente': 178.37919007682308,
 'te': 1675.3258637158917,
 'recuerdo': 306.48387045316315,
 'que': 2754.05210974035,
 'está': 1444.7053994278488,
 'aquí': 1349.9768500097691,
 'estaba': 996.2010785976058,
 'como': 2163.693302579083,
 'rabioso': 16.37494616013497,
 'con': 1854.840635821252,
 'especial': 503.554450076275,
 'lucas': 23.400066758614557,
 'martín': 80.14848223671673,
 'necesito': 322.01821137768957,
 'saber': 592.050234798179,
 'qué': 1366.1480148444505,
 'fue': 1524.2868763932865,
 'lo': 1667.7345204412673,
 'pasó': 526.0974232026272,
 'aplausos': 771.3768033282756}

Agregamos los vectores tfidf a cada subtítulo dentro del corpus

In [14]:
tfidf_obj.get_tfid_vectors()

Longitud de la oración más larga:  56


100%|██████████| 9/9 [00:06<00:00,  1.37it/s]


In [15]:
tfidf_obj.data.corpus[0][0]

{'id': 'L9YhoRatRzE',
 'original_title': 'Siempre Fui Yo | Adelanto | Disney+',
 'subtitles': [{'start': '0.13',
   'dur': '3.77',
   'text': ['tu', 'papá', 'tuvo', 'un', 'accidente'],
   'tfid': [1418.7040158067475,
    413.2296042991872,
    433.0538386324496,
    1977.6454572285645,
    178.37919007682308,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0]},
  {'start': '12.5',
   'dur': '5.939',
   'text': ['te', 'recuerdo', 'que', 'está'],
   'tfid': [1675.3258637158917,
    306.48387045316315,
    2754.05210974035,
    1444.7053994278488,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
   

## Recuperación de Documentos con TFIDF

Obtenemos los datos originales para regresar la cadena

In [16]:
videos_original = Data("../corpus/data")#Nota: no print de todo

In [17]:
def get_original(id, index_sub):
    '''
    Regresa el texto original

    Args:
        id (str): id del vídeo
        index_sub (int): índice del subtítulo del vídeo
    '''
    for chanel in videos_original.corpus:
        for video in chanel:
            if video['id'] == id:
                if 'subtitles' in video:
                    return video['subtitles'][index_sub]

In [18]:
def search(query, obj, n, prnt=False):
    '''
    Dada una query regresa el top n de los resultado en el corpus

    Args:
        query (str): frase/palabra a buscar
        obj (TFIDF): Objeto tfidf
        n     (int): número de resultados a regresar
    '''
    r = []
    results =  obj.recover_documents(query)
    #Ordena la consulta de mayor peso a menor
    sorted_results = sorted(results, key=itemgetter(1), reverse=True)[:n]
    for doc, score in sorted_results:
        info = get_original(doc[0], doc[1])
        r.append(info['text'])
        if prnt:
            print('Id: {} \nInfo: {}\n score: {}\n ---------'.format(doc[0],info, score))
    return r

Probamos la consulta con TF-IDF por subtítulo

In [19]:
#Consulta
query = 'osos polares comen trigo'
#Resultados de la consulta
r = search(query, tfidf_objxO, 5, True)

Id: yUJf5r7wq8g 
Info: {'start': '2126.62', 'dur': '6.05', 'text': 'el chupi dio en el trigo en la trampa'}
 score: 73956.34187723468
 ---------
Id: IV-8YsyghbU 
Info: {'start': '292.31', 'dur': '3.66', 'text': 'No se trata sólo de pequeños momentos de "ajá", como cuando descubrimos que los osos y los niños'}
 score: 67720.42831617984
 ---------
Id: 01lKDkYSFDg 
Info: {'start': '415.03', 'dur': '5.53', 'text': 'los cultivos que sobraban. Por décadas siguientes los agricultores de cultivos como el maíz, trigo, algodón y soja'}
 score: 67720.42831617984
 ---------
Id: Ru2SPCDVpWo 
Info: {'start': '1021.28', 'dur': '4.41', 'text': 'millones de toneladas anuales, Rusia es\xa0\nya el mayor exportador de trigo a Egipto,'}
 score: 64738.932663555366
 ---------
Id: zQLxEH_t6AM 
Info: {'start': '92.32', 'dur': '4.66', 'text': '[Y también del Kimchi, esa cosa roja que\nparece que los coreanos comen a todas horas'}
 score: 64738.932663555366
 ---------


Probamos la consulta con TF-IDF por video

In [21]:
#Consulta
query = 'osos polares comen trigo'
#Resultados de la consulta
r = search(query, tfidf_obj, 5, True)

Id: bx3CC_iZSRk 
Info: {'start': '56.95', 'dur': '4.86', 'text': 'se lo que comen'}
 score: 51.777104211023214
 ---------
Id: nLj0BV9dOiM 
Info: {'start': '14.139', 'dur': '1.167', 'text': '...y comen mucho.'}
 score: 43.60177196717744
 ---------
Id: HFrcY9dzmFg 
Info: {'start': '122.747', 'dur': '3.462', 'text': 'pero, lo siento,\nmi mamá es alérgica a los osos.'}
 score: 21.252941393875044
 ---------
Id: nLj0BV9dOiM 
Info: {'start': '8.925', 'dur': '5.005', 'text': 'Además, estos osos duermen todo el día...'}
 score: 21.252941393875044
 ---------
Id: YuJDV1ewv24 
Info: {'start': '5.083', 'dur': '3.208', 'text': '¡Oh, no! Se comen\ntodos los donuts con jalea.'}
 score: 20.243679841903813
 ---------


## Probamos con el conjunto test
...para posteriormente evaluarlo

In [22]:
with open("../pkl/test.pkl", "rb") as f:
    test = pickle.load(f)

Probamos la consulta con TF-IDF por subtítulo

In [ ]:
#Consulta
test_results = []

for query in tqdm(test):
    r = search(query, tfidf_objxO, 5)
    test_results.append(r)

pickle.dump(test_results, open("../pkl/resultsxOracion.pkl", "wb"))

In [25]:
with open("../pkl/resultsxOracion.pkl", "rb") as f:
    test_resultsxO = pickle.load(f)

for i in range(2):
    print(test[i], test_resultsxO[i])
    print("-------------")

a un lado oigan alejen a los demás viene ['dejéis a un lado los fanatismos no tiene', 'hare a un lado la batidora pues es momento\nde añadir los ingredientes secos que tamizamos.', 'Dejare a un lado mi batidora pues ire\nagregando los ingredientes secos.', 'Dejare a un lado mi batidora e ire\nincorporando los ingredientes secos...', 'Pon a un lado tu batidora y vacia \nlos ingredientes secos.']
-------------
saber lo que se está toda máquina ['saber lo que se está toda máquina', 'creo que se lo está perdiendo', 'ver que la gente se lo está creyendo y', 'bebé aún se alimenta de lo que está', 'es algo que se está perdiendo y si lo']
-------------


Probamos la consulta con TF-IDF por video

In [ ]:
#Consulta
test_results = []

for query in tqdm(test):
    r = search(query, tfidf_objxO, 5)
    test_results.append(r)

pickle.dump(test_results, open("../pkl/results.pkl", "wb"))

In [27]:
with open("../pkl/results.pkl", "rb") as f:
    test_results = pickle.load(f)

for i in range(2):
    print(test[i], test_results[i])
    print("-------------")

a un lado oigan alejen a los demás viene ['Vámonos a casa.', '¿Cómo ibas a saberlo?', '¡Hola a todos!', 'Son fisuras\na un infierno desconocido.', 'y fue porque íbamos a saltar de los']
-------------
saber lo que se está toda máquina ['...que nada.', 'No hasta que te derroque', 'No dejes que el fuego te controle.', 'Sé que nunca pediste esto.', 'necesito saber qué fue lo que pasó']
-------------
